In [1]:
#https://www.kaggle.com/kanncaa1/recommendation-systems-tutorial/data

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy as scipy
import math as math
import random as random
import sklearn

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds

%matplotlib inline

In [3]:
movies_df = pd.read_csv('movie.csv')

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_df['title_1'] = movies_df['title'].str.split("\(\d", expand=True)[0]
movies_df['year'] = movies_df['title'].str.extract('.*\((.*\d{4})\).*', expand=True)
movies_df.head(30)

,movieId,title,genres,title_1,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II,1995
5,6,Heat (1995),Action|Crime|Thriller,Heat,1995
6,7,Sabrina (1995),Comedy|Romance,Sabrina,1995
7,8,Tom and Huck (1995),Adventure|Children,Tom and Huck,1995
8,9,Sudden Death (1995),Action,Sudden Death,1995
9,10,GoldenEye (1995),Action|Adventure|Thriller,GoldenEye,1995


In [6]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 5 columns):
movieId    27278 non-null int64
title      27278 non-null object
genres     27278 non-null object
title_1    27278 non-null object
year       27257 non-null object
dtypes: int64(1), object(4)
memory usage: 1.0+ MB


In [7]:
interactions_df = pd.read_csv('rating.csv')
interactions_df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [8]:
interactions_df.drop('timestamp', axis=1, inplace=True)

In [9]:
interactions_df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [10]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
userId     int64
movieId    int64
rating     float64
dtypes: float64(1), int64(2)
memory usage: 457.8 MB


In [11]:
interactions_df = interactions_df.iloc[:20000,:]

In [12]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 3 columns):
userId     20000 non-null int64
movieId    20000 non-null int64
rating     20000 non-null float64
dtypes: float64(1), int64(2)
memory usage: 468.8 KB


In [13]:
print(interactions_df['userId'].nunique())
print(interactions_df['movieId'].nunique())

156
4192


In [14]:
## Merge movies_df (with only movieId, title columns) with interactions_df

interactions_movies_df = interactions_df.merge(movies_df[['movieId', 'title_1', 'year']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')

In [15]:
interactions_movies_df.head(30)

,userId,movieId,rating,title_1,year
0,1,2,3.5,Jumanji,1995
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995
3,1,47,3.5,Seven (a.k.a. Se7en),1995
4,1,50,3.5,"Usual Suspects, The",1995
5,1,112,3.5,Rumble in the Bronx (Hont faan kui),1995
6,1,151,4.0,Rob Roy,1995
7,1,223,4.0,Clerks,1994
8,1,253,4.0,Interview with the Vampire: The Vampire Chroni...,1994
9,1,260,4.0,Star Wars: Episode IV - A New Hope,1977


In [16]:
interactions_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 0 to 19999
Data columns (total 5 columns):
userId     20000 non-null int64
movieId    20000 non-null int64
rating     20000 non-null float64
title_1    20000 non-null object
year       20000 non-null object
dtypes: float64(1), int64(2), object(2)
memory usage: 937.5+ KB


## Popularity Model

In [17]:
#interactions_movies_df[interactions_movies_df['movieId']==118696]['rating'].avg()

interactions_movies_grp = interactions_movies_df.groupby(['movieId', 'title_1']).agg({'rating': 'sum'}).reset_index()
interactions_movies_grp.rename(columns = {'rating': 'TotalRating'},inplace=True)
interactions_movies_grp.head()

,movieId,title_1,TotalRating
0,1,Toy Story,221.5
1,2,Jumanji,60.0
2,3,Grumpier Old Men,59.5
3,4,Waiting to Exhale,8.0
4,5,Father of the Bride Part II,35.5


In [18]:
#Sort movie_Id based upon rating

popular_ratings_df = interactions_movies_grp.sort_values(['TotalRating', 'title_1', 'movieId'], ascending=[0,1,1])
popular_ratings_df.head(10)

,movieId,title_1,TotalRating
241,296,Pulp Fiction,339.5
289,356,Forrest Gump,324.0
258,318,"Shawshank Redemption, The",312.5
463,593,"Silence of the Lambs, The",273.0
379,480,Jurassic Park,266.5
209,260,Star Wars: Episode IV - A New Hope,231.0
472,608,Fargo,228.5
45,50,"Usual Suspects, The",228.5
1859,2858,American Beauty,223.0
0,1,Toy Story,221.5


In [19]:
#Generate a recommendation rank based upon score
popular_ratings_df['Rank'] = popular_ratings_df['TotalRating'].rank(ascending=0, method='first')
popular_ratings_df.head()

,movieId,title_1,TotalRating,Rank
241,296,Pulp Fiction,339.5,1.0
289,356,Forrest Gump,324.0,2.0
258,318,"Shawshank Redemption, The",312.5,3.0
463,593,"Silence of the Lambs, The",273.0,4.0
379,480,Jurassic Park,266.5,5.0


In [20]:
#Get the top 10 recommendations
popularity_recommendations = popular_ratings_df.head(10)
popularity_recommendations

,movieId,title_1,TotalRating,Rank
241,296,Pulp Fiction,339.5,1.0
289,356,Forrest Gump,324.0,2.0
258,318,"Shawshank Redemption, The",312.5,3.0
463,593,"Silence of the Lambs, The",273.0,4.0
379,480,Jurassic Park,266.5,5.0
209,260,Star Wars: Episode IV - A New Hope,231.0,6.0
472,608,Fargo,228.5,7.0
45,50,"Usual Suspects, The",228.5,8.0
1859,2858,American Beauty,223.0,9.0
0,1,Toy Story,221.5,10.0


#### Define a Popularity Recommender class with the above code

In [21]:
class popularity_recommender_py():
    def __init__(self):
        self.data = None
        self.rating = None
        self.movieId = None
        self.userId = None        
        self.title_1 = None
        self.popularity_recommendations = None
        
    #Create the popularity based recommender system model
    def create(self, data, movieId, title_1, rating):
        self.data = data        
        self.movieId = movieId
        self.title_1 = title_1
        self.rating = rating

        df_grp = data.groupby(['movieId', 'title_1']).agg({'rating': 'sum'}).reset_index()
        df_grp.rename(columns = {'rating': 'TotalRating'},inplace=True)

        #Sort the movieId's based upon recommendation score
        popular_movies_df = df_grp.sort_values(['TotalRating', 'title_1', 'movieId'], ascending=[0,1,1])
        
        #Generate a recommendation rank based upon score
        popular_movies_df['Rank'] = popular_movies_df['TotalRating'].rank(ascending=0, method='first')
        
        #Get the top 10 recommendations
        self.popularity_recommendations = popular_movies_df.head(10)

        
    #Use the popularity based recommender system model to make recommendations of contents that the user has not interacted with:
    def recommend(self, userId, items_to_ignore=[], topn=10):    
        user_recommendations = self.popularity_recommendations[~self.popularity_recommendations['movieId'].isin(items_to_ignore)].head(topn)
        
        #Add userId column for which the recommendations are being generated
        user_recommendations['userId'] = userId
    
        #Bring userId column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]
        
        return user_recommendations
       

In [22]:
## Before we call the class and its functions, lets first create the unique personIds

In [23]:
## Get unique personIds
users = interactions_movies_df['userId'].unique()
users

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156], dtype=int64)

In [24]:
#interactions_movies_df.head()

In [25]:
## Create an object for the class and instantiate the functions within the class
pm = popularity_recommender_py()
pm.create(interactions_movies_df, 'movieId', 'title_1', 'rating')

In [26]:
user_id = users[3]

## get list of already interacted content ids (in order to avoid coming up in the popularity recommendation list)
items_to_ignore = interactions_movies_df[interactions_movies_df['userId'] == user_id]['movieId'].tolist()
#items_to_ignore

pm.recommend(user_id,items_to_ignore)

,userId,movieId,title_1,TotalRating,Rank
241,4,296,Pulp Fiction,339.5,1.0
258,4,318,"Shawshank Redemption, The",312.5,3.0
463,4,593,"Silence of the Lambs, The",273.0,4.0
209,4,260,Star Wars: Episode IV - A New Hope,231.0,6.0
472,4,608,Fargo,228.5,7.0
45,4,50,"Usual Suspects, The",228.5,8.0
1859,4,2858,American Beauty,223.0,9.0
0,4,1,Toy Story,221.5,10.0


In [27]:
user_id = users[30]

## get list of already interacted content ids (in order to avoid coming up in the popularity recommendation list)
items_to_ignore = interactions_movies_df[interactions_movies_df['userId'] == user_id]['movieId'].tolist()
#items_to_ignore

pm.recommend(user_id,items_to_ignore)

,userId,movieId,title_1,TotalRating,Rank
241,31,296,Pulp Fiction,339.5,1.0
289,31,356,Forrest Gump,324.0,2.0
258,31,318,"Shawshank Redemption, The",312.5,3.0
463,31,593,"Silence of the Lambs, The",273.0,4.0
379,31,480,Jurassic Park,266.5,5.0
472,31,608,Fargo,228.5,7.0
45,31,50,"Usual Suspects, The",228.5,8.0
1859,31,2858,American Beauty,223.0,9.0


## Content based Filtering model

In [28]:
## Content-based filtering approaches leverage description or attributes from items the user has interacted 
## to recommend similar items.
## It depends only on the user's previous choices, making this method robust to avoid the cold-start problem. 

In [29]:
#Ignoring stopwords (words with no semantics) from English
stopwords_list = stopwords.words('english')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range= (1,2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

In [30]:
#title_genre = pd.concat([movies_df['Title'], movies_df['Genre']])
#title_genre = movies_df['Title'] + " " + movies_df['Genre']
genre = movies_df['genres']

In [31]:
genre

0        Adventure|Animation|Children|Comedy|Fantasy
1                         Adventure|Children|Fantasy
2                                     Comedy|Romance
3                               Comedy|Drama|Romance
4                                             Comedy
5                              Action|Crime|Thriller
6                                     Comedy|Romance
7                                 Adventure|Children
8                                             Action
9                          Action|Adventure|Thriller
10                              Comedy|Drama|Romance
11                                     Comedy|Horror
12                      Adventure|Animation|Children
13                                             Drama
14                          Action|Adventure|Romance
15                                       Crime|Drama
16                                     Drama|Romance
17                                            Comedy
18                                            

In [32]:
tfidf_matrix = vectorizer.fit_transform(genre)

In [33]:
tfidf_feature_names = vectorizer.get_feature_names()

In [34]:
print(len(tfidf_feature_names))
print(tfidf_feature_names)

90
['action', 'action adventure', 'action animation', 'action comedy', 'action crime', 'action drama', 'action horror', 'action sci', 'action thriller', 'adventure', 'adventure animation', 'adventure children', 'adventure comedy', 'adventure crime', 'adventure drama', 'adventure fantasy', 'adventure sci', 'animation', 'animation children', 'animation comedy', 'children', 'children comedy', 'children drama', 'children fantasy', 'comedy', 'comedy crime', 'comedy documentary', 'comedy drama', 'comedy fantasy', 'comedy horror', 'comedy musical', 'comedy romance', 'comedy sci', 'comedy western', 'crime', 'crime drama', 'crime horror', 'crime mystery', 'crime thriller', 'documentary', 'documentary drama', 'documentary musical', 'drama', 'drama fantasy', 'drama film', 'drama horror', 'drama musical', 'drama mystery', 'drama romance', 'drama sci', 'drama thriller', 'drama war', 'drama western', 'fantasy', 'fantasy horror', 'fantasy musical', 'fantasy mystery', 'fantasy romance', 'fantasy sci',

In [35]:
item_ids = movies_df['movieId'].tolist()
#item_ids.index(-8949113594875411859)
#item_ids

In [36]:
interactions_movies_df.head()

,userId,movieId,rating,title_1,year
0,1,2,3.5,Jumanji,1995
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995
3,1,47,3.5,Seven (a.k.a. Se7en),1995
4,1,50,3.5,"Usual Suspects, The",1995


In [37]:
users_movies_df = interactions_movies_df.set_index('userId')

In [38]:
users_movies_df.head()

,movieId,rating,title_1,year
userId,,,,
1,2,3.5,Jumanji,1995
1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995
1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995
1,47,3.5,Seven (a.k.a. Se7en),1995
1,50,3.5,"Usual Suspects, The",1995


In [39]:
users_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 156
Data columns (total 4 columns):
movieId    20000 non-null int64
rating     20000 non-null float64
title_1    20000 non-null object
year       20000 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 781.2+ KB


In [40]:
users_movies_df.dropna(inplace=True)

In [41]:
users_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000 entries, 1 to 156
Data columns (total 4 columns):
movieId    20000 non-null int64
rating     20000 non-null float64
title_1    20000 non-null object
year       20000 non-null object
dtypes: float64(1), int64(1), object(2)
memory usage: 781.2+ KB


#### To model the user profile, we take all the item profiles the user has interacted with and average them.

#### The average is weighted by the rating

In [42]:
users_movies_df.groupby([users_movies_df.index])[['movieId']].count().sort_values(by='movieId', ascending=True).head(5)


,movieId
userId,
36,20
52,20
39,20
37,20
123,22


In [43]:
users_movies_df.loc[36].sort_values(by='rating', ascending=False)

,movieId,rating,title_1,year
userId,,,,
36,2353,4.0,Enemy of the State,1998
36,145,3.5,Bad Boys,1995
36,1358,3.5,Sling Blade,1996
36,2023,3.5,"Godfather: Part III, The",1990
36,1408,3.5,"Last of the Mohicans, The",1992
36,1597,3.5,Conspiracy Theory,1997
36,58293,3.5,"10,000 BC",2008
36,1088,3.0,Dirty Dancing,1987
36,163,3.0,Desperado,1995


In [44]:
def get_user_profiles():
    user_profiles = {}
    
    ## Create an index_df by joinning the interactions_full_df with the articles_df, and get the columns from the interactions_full_df
    #interactions_indexed_df = interactions_full_df[interactions_articles_df['contentId'].isin(articles_df['contentId'])].set_index('personId')

    ## For every user in the index_df, get all of their contentIds, and get their corresponding tfdifmatrix 

    for userid in users_movies_df.index.unique():
        interactions_person_df = users_movies_df.loc[userid]    
        user_item_strengths = np.array(interactions_person_df['rating']).reshape(-1,1) ## n rows x 1 col
    
        itemprofilelist = [tfidf_matrix[item_ids.index(c)] for c in pd.Series(interactions_person_df['movieId'])]
        item_profiles = scipy.sparse.vstack(itemprofilelist)
    
        #Weighted average of item profiles by the interactions strength
        user_item_strengths_weighted_avg = np.sum(item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
        user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)    
        
        user_profiles[userid] = user_profile_norm
    return user_profiles

In [45]:
user_profiles = get_user_profiles()

In [46]:
len(user_profiles)

156

In [47]:
## Let's take a look in the profile. It is a unit vector of length 90 (length of tf-idf matrix). 
#The value in each position represents how relevant is a token (unigram or bigram).

In [48]:
myprofile = user_profiles[36]
print(myprofile.shape)

(1, 90)


In [49]:
myprofile.flatten().tolist()

[0.26329893543006716,
 0.027555627256750648,
 0.0,
 0.07462009815710449,
 0.0,
 0.0,
 0.056146109767782414,
 0.0,
 0.1509466724348748,
 0.09268116740707781,
 0.0,
 0.0,
 0.03341814887706501,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3737383884535064,
 0.13995271933289602,
 0.0,
 0.0,
 0.08358623875789668,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.19138174846036363,
 0.11733353216192162,
 0.0,
 0.0,
 0.07135729712672709,
 0.0,
 0.0,
 0.0,
 0.4082476502008344,
 0.0,
 0.0,
 0.0,
 0.07944279230360352,
 0.24197265230443296,
 0.0,
 0.0,
 0.06273771358665198,
 0.0,
 0.0,
 0.0627395786586514,
 0.0,
 0.0,
 0.0,
 0.05693933159937356,
 0.0,
 0.09063894076818212,
 0.0453409145933734,
 0.0,
 0.0,
 0.0,
 0.0,
 0.08087551236166499,
 0.0,
 0.12616941338044776,
 0.0,
 0.0,
 0.0,
 0.06196542448391175,
 0.07889335612806661,
 0.19024550168531013,
 0.08651119051499069,
 0.0,
 0.16399717686689624,
 0.0,
 0.0,
 0.2602556606661072,
 0.0,
 0.19663651734251553,
 0.09970130483052396,
 0.0985651

In [50]:
token_relevance = pd.DataFrame(sorted(zip(tfidf_feature_names,user_profiles[36].flatten().tolist())), columns=['token', 'relevance']) 

In [51]:
token_relevance.head(20)

,token,relevance
0,action,0.263299
1,action adventure,0.027556
2,action animation,0.000000
3,action comedy,0.074620
4,action crime,0.000000
5,action drama,0.000000
6,action horror,0.056146
7,action sci,0.000000
8,action thriller,0.150947
9,adventure,0.092681


In [52]:
## Sort by highest order of relevance

token_relevance = token_relevance.sort_values(by='relevance', ascending=False)

In [53]:
token_relevance.head(30)

,token,relevance
42,drama,0.408248
86,thriller,0.376396
24,comedy,0.373738
0,action,0.263299
79,romance,0.260256
47,drama mystery,0.241973
81,romance thriller,0.196637
34,crime,0.191382
73,mystery,0.190246
76,mystery thriller,0.163997


In [54]:
#Compute the cosine similarity between the user profile and all item profiles
person_id = 36
cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
cosine_similarities 

array([[ 0.13811655,  0.05289015,  0.27349082, ...,  0.09268117,
         0.        ,  0.07990969]])

In [55]:
cosine_similarities.shape

(1, 27278)

In [56]:
#Gets the top similar items
topn = 15
similar_indices = cosine_similarities.argsort().flatten()[-topn:]
similar_indices

array([14157,  1712,  6841,  2717,  9038, 24334,  9791, 16419,  1726,
       24335, 24336,  8611, 12003, 24664,  9422], dtype=int64)

In [57]:
#Sort the similar items by similarity
similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
similar_items

[(82915, 0.63002040523487368),
 (1799, 0.63002040523487368),
 (115335, 0.63002040523487368),
 (115337, 0.63002040523487368),
 (26104, 0.63002040523487368),
 (54262, 0.63002040523487368),
 (116698, 0.63002040523487368),
 (27674, 0.63002040523487368),
 (115333, 0.58738668892917878),
 (31921, 0.58738668892917878),
 (71033, 0.57836218157594188),
 (1783, 0.57836218157594188),
 (6953, 0.57836218157594188),
 (2803, 0.57836218157594188),
 (26761, 0.57836218157594188)]

In [58]:
items_to_ignore = interactions_movies_df[interactions_movies_df['userId'] == person_id]['movieId'].tolist()
items_to_ignore

[145,
 163,
 196,
 485,
 555,
 762,
 1088,
 1320,
 1358,
 1408,
 1597,
 1625,
 1673,
 2023,
 2100,
 2302,
 2353,
 2712,
 2770,
 58293]

In [59]:
similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items)) ##x[0] is the contentId, x[1] is cosine similarity
len(similar_items_filtered)

15

In [60]:
similar_items_filtered

[(82915, 0.63002040523487368),
 (1799, 0.63002040523487368),
 (115335, 0.63002040523487368),
 (115337, 0.63002040523487368),
 (26104, 0.63002040523487368),
 (54262, 0.63002040523487368),
 (116698, 0.63002040523487368),
 (27674, 0.63002040523487368),
 (115333, 0.58738668892917878),
 (31921, 0.58738668892917878),
 (71033, 0.57836218157594188),
 (1783, 0.57836218157594188),
 (6953, 0.57836218157594188),
 (2803, 0.57836218157594188),
 (26761, 0.57836218157594188)]

In [61]:
content_rec_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'recStrength']).head(topn)

In [62]:
content_rec_df

,movieId,recStrength
0,82915,0.630020
1,1799,0.630020
2,115335,0.630020
3,115337,0.630020
4,26104,0.630020
5,54262,0.630020
6,116698,0.630020
7,27674,0.630020
8,115333,0.587387
9,31921,0.587387


In [63]:
## Merge interactions_movies_df with rec_df

content_rec_df_completed = content_rec_df.merge(movies_df[['movieId', 'title', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')

In [64]:
## Recommended content for personId: 36
content_rec_df_completed

,movieId,recStrength,title,genres
0,82915,0.630020,"Tiger's Tail, The (2006)",Comedy|Crime|Drama|Mystery|Thriller
1,1799,0.630020,Suicide Kings (1997),Comedy|Crime|Drama|Mystery|Thriller
2,115335,0.630020,Charlie Chan in Paris (1935),Comedy|Crime|Drama|Mystery|Thriller
3,115337,0.630020,Charlie Chan in Reno (1939),Comedy|Crime|Drama|Mystery|Thriller
4,26104,0.630020,Murder at the Gallop (1963),Comedy|Crime|Drama|Mystery|Thriller
5,54262,0.630020,Murder Most Foul (1964),Comedy|Crime|Drama|Mystery|Thriller
6,116698,0.630020,Dead Men Tell (1941),Comedy|Crime|Drama|Mystery|Thriller
7,27674,0.630020,11:14 (2003),Comedy|Crime|Drama|Mystery|Thriller
8,115333,0.587387,Charlie Chan in Panama (1940),Adventure|Comedy|Crime|Drama|Mystery|Thriller
9,31921,0.587387,"Seven-Per-Cent Solution, The (1976)",Adventure|Comedy|Crime|Drama|Mystery|Thriller


### Define the above in a class

In [65]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def get_similar_items_to_user_profile(self, user_id, topn=1000):
        
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[user_id], tfidf_matrix)
        
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        
        return similar_items
        
    def content_recommend_items(self, user_id, articles_df, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self.get_similar_items_to_user_profile(user_id)
        
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        rec_df = pd.DataFrame(similar_items_filtered, columns=['movieId', 'recStrength']).head(topn)

       
       # if verbose:
       #     if self.items_df is None:
       #         raise Exception('"items_df" is required in verbose mode')

       #     recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
       #                                                   left_on = 'contentId', 
       #                                                   right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]
       

        content_recommendation_df = rec_df.merge(movies_df[['movieId', 'title', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')
    
        return content_recommendation_df

In [66]:
cont_rec = ContentBasedRecommender()

In [67]:
user_id = 36

## get list of already interacted content ids (in order to avoid coming up in the recommendation list)
ignore_items = interactions_movies_df[interactions_movies_df['userId'] == user_id]['movieId'].tolist()

recommendation_df = cont_rec.content_recommend_items(user_id, movies_df, ignore_items)

In [68]:
recommendation_df

,movieId,recStrength,title,genres
0,82915,0.630020,"Tiger's Tail, The (2006)",Comedy|Crime|Drama|Mystery|Thriller
1,1799,0.630020,Suicide Kings (1997),Comedy|Crime|Drama|Mystery|Thriller
2,115335,0.630020,Charlie Chan in Paris (1935),Comedy|Crime|Drama|Mystery|Thriller
3,115337,0.630020,Charlie Chan in Reno (1939),Comedy|Crime|Drama|Mystery|Thriller
4,26104,0.630020,Murder at the Gallop (1963),Comedy|Crime|Drama|Mystery|Thriller
5,54262,0.630020,Murder Most Foul (1964),Comedy|Crime|Drama|Mystery|Thriller
6,116698,0.630020,Dead Men Tell (1941),Comedy|Crime|Drama|Mystery|Thriller
7,27674,0.630020,11:14 (2003),Comedy|Crime|Drama|Mystery|Thriller
8,115333,0.587387,Charlie Chan in Panama (1940),Adventure|Comedy|Crime|Drama|Mystery|Thriller
9,31921,0.587387,"Seven-Per-Cent Solution, The (1976)",Adventure|Comedy|Crime|Drama|Mystery|Thriller


In [127]:
interactions_movies_df[interactions_movies_df['userId'] == 1][['rating', 'title_1', 'genres']].sort_values(by='rating', ascending=False)

KeyError: "['genres'] not in index"

In [123]:
user_id = 1

## get list of already interacted content ids (in order to avoid coming up in the recommendation list)
ignore_items = interactions_movies_df[interactions_movies_df['userId'] == user_id]['movieId'].tolist()

recommendation_df = cont_rec.content_recommend_items(user_id, movies_df, ignore_items)
recommendation_df

,movieId,recStrength,title,genres
0,1562,0.695340,Batman & Robin (1997),Action|Adventure|Fantasy|Thriller
1,61818,0.695340,"Crow, The: Wicked Prayer (2005)",Action|Adventure|Fantasy|Thriller
2,60937,0.695340,"Mummy: Tomb of the Dragon Emperor, The (2008)",Action|Adventure|Fantasy|Thriller
3,4781,0.679216,Megiddo: The Omega Code 2 (2001),Action|Adventure|Fantasy|Sci-Fi|Thriller
4,74619,0.673114,Mysterious Island (2005),Action|Adventure|Fantasy|Sci-Fi
5,123565,0.673114,Hercules in the Underworld (1994),Action|Adventure|Fantasy|Sci-Fi
6,123159,0.673114,Hercules and the Lost Kingdom (1994),Action|Adventure|Fantasy|Sci-Fi
7,121117,0.673114,Highlander: The Source (2007),Action|Adventure|Fantasy|Sci-Fi
8,53453,0.673114,Starcrash (a.k.a. Star Crash) (1978),Action|Adventure|Fantasy|Sci-Fi
9,87433,0.673114,"Ghidorah, the Three-Headed Monster (San daikai...",Action|Adventure|Fantasy|Sci-Fi


### Collaborative Filtering - Matrix Factorization

In [69]:
interactions_movies_df.head()

,userId,movieId,rating,title_1,year
0,1,2,3.5,Jumanji,1995
1,1,29,3.5,"City of Lost Children, The (Cité des enfants p...",1995
2,1,32,3.5,Twelve Monkeys (a.k.a. 12 Monkeys),1995
3,1,47,3.5,Seven (a.k.a. Se7en),1995
4,1,50,3.5,"Usual Suspects, The",1995


In [70]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_movies_df.pivot(index='userId', 
                                                          columns='movieId', 
                                                          values='rating').fillna(0)

users_items_pivot_matrix_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,112623,112852,113453,114180,115617,116797,117511,117590,118696,125916
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
users_items_pivot_matrix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 1 to 156
Columns: 4192 entries, 1 to 125916
dtypes: float64(4192)
memory usage: 5.0 MB


In [72]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix#[:10]

array([[ 0. ,  3.5,  0. , ...,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  4. , ...,  0. ,  0. ,  0. ],
       [ 4. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       ..., 
       [ 0. ,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 2.5,  0. ,  0. , ...,  0. ,  0. ,  0. ],
       [ 5. ,  5. ,  2. , ...,  0. ,  0. ,  0. ]])

In [73]:
type(users_items_pivot_matrix)

numpy.ndarray

In [74]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [75]:
## An important decision is the number of factors to factor the user-item matrix. 
## The higher the number of factors, the more precise is the factorization in the original matrix reconstructions.
## Therefore, if the model is allowed to memorize too much details of the original matrix,
## it may not generalize well for data it was not trained on.

## Reducing the number of factors increases the model generalization.

In [76]:
users_items_pivot_matrix.shape

(156, 4192)

In [77]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 50

In [78]:
#Performs matrix factorization of the original user item matrix

U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [79]:
U.shape

(156, 50)

In [80]:
Vt.shape

(50, 4192)

In [81]:
sigma.shape

(50,)

In [82]:
U

array([[ 0.06184608, -0.07561962, -0.1326474 , ..., -0.06874625,
         0.01536585, -0.08690391],
       [-0.1224761 ,  0.06587255,  0.01388649, ...,  0.0107948 ,
        -0.00243868, -0.03035203],
       [-0.05357331, -0.02342385,  0.0490926 , ..., -0.02890714,
        -0.09670381, -0.11649544],
       ..., 
       [ 0.01936179, -0.0553438 , -0.02622001, ..., -0.07263616,
         0.1027582 , -0.10902579],
       [ 0.06874246,  0.02598165,  0.04410185, ..., -0.04825626,
         0.05564652, -0.04130929],
       [-0.18835127,  0.16764721, -0.20860409, ...,  0.17155217,
        -0.01396574, -0.05657517]])

In [83]:
Vt

array([[ -1.23443977e-02,  -3.16029780e-03,  -4.37576214e-02, ...,
         -8.73536343e-03,  -6.70676759e-04,  -6.70676759e-05],
       [ -3.10661528e-02,   7.08670087e-03,   1.84381356e-02, ...,
         -4.02757629e-03,   4.58974420e-03,   4.58974420e-04],
       [ -3.87846240e-02,  -2.18840263e-02,  -1.82633691e-02, ...,
          5.74398536e-03,  -1.34558381e-03,  -1.34558381e-04],
       ..., 
       [  4.09756142e-02,   4.20628960e-02,   2.24942645e-02, ...,
         -6.50838871e-04,  -4.13775446e-03,  -4.13775446e-04],
       [  3.56152238e-02,   9.99248804e-03,   1.57928098e-02, ...,
          1.10826248e-03,   9.01015629e-03,   9.01015629e-04],
       [ -8.22184559e-02,  -2.64429039e-02,  -1.57220361e-02, ...,
         -2.64813234e-04,  -1.04255146e-03,  -1.04255146e-04]])

In [84]:
sigma

array([  36.17382235,   36.98331046,   37.19645062,   37.35258364,
         37.57659011,   38.49772945,   38.58295752,   38.72130869,
         39.23356808,   39.53820397,   40.03450547,   40.19369613,
         41.19267016,   41.43664192,   41.81283582,   42.84103222,
         43.12415354,   43.71854792,   44.01151045,   44.4968366 ,
         45.05611161,   45.70236258,   46.22865529,   46.93254952,
         47.80637316,   49.24483286,   49.78441616,   50.9655816 ,
         51.37703739,   51.60122987,   52.95977458,   54.29886252,
         54.76973053,   56.52413846,   57.48302832,   57.67154794,
         59.83429261,   61.42398611,   62.60858681,   64.05275921,
         66.79066452,   70.23313468,   71.87338517,   74.61942588,
         78.13077042,   83.6833512 ,   86.36045679,  104.30674901,
        111.5960118 ,  230.91129824])

In [85]:
sigma = np.diag(sigma)
sigma.shape

(50, 50)

In [86]:
sigma

array([[  36.17382235,    0.        ,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,   36.98331046,    0.        , ...,    0.        ,
           0.        ,    0.        ],
       [   0.        ,    0.        ,   37.19645062, ...,    0.        ,
           0.        ,    0.        ],
       ..., 
       [   0.        ,    0.        ,    0.        , ...,  104.30674901,
           0.        ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
         111.5960118 ,    0.        ],
       [   0.        ,    0.        ,    0.        , ...,    0.        ,
           0.        ,  230.91129824]])

In [87]:
## After the factorization, we try to to reconstruct the original matrix by multiplying its factors. 
## The resulting matrix is not sparse any more.

## It generated predictions for items the user has not yet interacted, which we will exploit for recommendations.

In [88]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)

all_user_predicted_ratings

array([[  5.99611204e-01,   3.26258524e+00,  -1.01087277e-01, ...,
         -6.07326396e-02,  -1.88358305e-02,  -1.88358305e-03],
       [  6.98746992e-01,   3.80624901e-01,   1.03088057e+00, ...,
         -3.43261582e-02,   1.05431321e-01,   1.05431321e-02],
       [  4.45323051e+00,   3.29654909e-03,  -3.12973702e-01, ...,
          9.15177228e-02,   7.73604333e-03,   7.73604333e-04],
       ..., 
       [  4.14455070e-01,  -1.91356543e-01,   2.16320194e-01, ...,
         -7.49147730e-02,  -3.12321445e-02,  -3.12321445e-03],
       [  7.48480431e-01,   2.65448178e-01,   3.96517707e-01, ...,
         -3.09586190e-02,   7.45271486e-02,   7.45271486e-03],
       [  4.75433562e+00,   4.13540551e+00,   2.15061424e+00, ...,
          3.28790885e-02,  -3.72129769e-02,  -3.72129769e-03]])

In [89]:
all_user_predicted_ratings.shape

(156, 4192)

In [90]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,1,2,3,4,5,6,7,8,9,10,...,147,148,149,150,151,152,153,154,155,156
movieId,,,,,,,,,,,,,,,,,,,,,
1,0.599611,0.698747,4.453231,0.679727,2.168952,3.296606,-0.310101,1.742570,0.005556,0.681066,...,4.760476,0.853245,1.762139,0.025119,0.418810,0.320853,-0.181354,0.414455,0.748480,4.754336
2,3.262585,0.380625,0.003297,0.289157,0.499934,0.287561,0.495997,1.193663,0.083512,-0.206406,...,-0.029974,-0.491637,0.645078,0.114302,0.161090,1.055905,0.059698,-0.191357,0.265448,4.135406
3,-0.101087,1.030881,-0.312974,0.161806,1.585538,2.117141,3.172050,0.717718,0.023801,0.235253,...,0.076916,-0.110658,0.397496,0.028003,0.148002,-0.176604,-0.015627,0.216320,0.396518,2.150614
4,-0.186174,0.220043,0.069618,-0.216058,-0.381269,0.139844,0.061206,-0.017122,0.007479,-0.082773,...,-0.084542,-0.025495,-0.147291,0.072158,-0.083188,0.058978,0.150028,0.026279,0.242424,2.179800
5,-0.061950,0.364409,-0.320773,0.240251,0.462498,0.827814,-0.022522,0.578909,0.008623,-0.138768,...,0.168842,-0.090138,0.223516,0.031387,0.145750,-0.461182,-0.073557,0.158827,-0.063693,2.002719
6,-0.430448,0.850223,-0.120608,0.441755,0.972962,1.726309,-0.313508,0.959346,-0.059669,0.280324,...,3.839314,0.666961,0.052517,-0.069954,0.146876,-0.227345,0.272619,5.293265,0.844549,4.311804
7,0.117004,0.613558,-0.177138,-0.137557,1.051998,2.391056,3.574820,0.129490,-0.015731,-0.037278,...,-0.005258,3.254430,0.166188,-0.121475,0.119660,-0.671689,-0.351043,-0.253093,0.201336,4.190984
8,-0.003157,0.035250,-0.024831,0.038769,0.094140,0.030632,0.059857,0.146352,0.034943,0.019137,...,-0.005716,0.118346,0.037616,-0.007123,0.024990,-0.020638,0.033221,-0.045760,-0.033029,0.125666
9,0.117493,0.231167,0.032385,0.133503,-0.143718,0.110620,-0.050339,0.663361,-0.050998,0.138656,...,0.029639,-0.242202,0.065592,-0.021912,0.135259,-0.449867,-0.477942,-0.071198,0.077894,3.374176


In [91]:
cf_preds_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4192 entries, 1 to 125916
Columns: 156 entries, 1 to 156
dtypes: float64(156)
memory usage: 5.2 MB


In [92]:
userid = 36

cf_preds_df[userid].sort_values(ascending=False).head()

movieId
2353    0.496649
555     0.480437
1358    0.412831
1466    0.411030
1625    0.379097
Name: 36, dtype: float64

In [93]:
sorted_user_predictions = cf_preds_df[userid].sort_values(ascending=False).reset_index().rename(columns={userid: 'recStrength'})
sorted_user_predictions

,movieId,recStrength
0,2353,0.496649
1,555,0.480437
2,1358,0.412831
3,1466,0.411030
4,1625,0.379097
5,2028,0.371636
6,1527,0.351813
7,1722,0.339048
8,1777,0.338855
9,163,0.334001


In [94]:
# Recommend the highest predicted rating movies that the user hasn't seen yet.

## get list of already interacted content ids (in order to avoid coming up in the recommendation list)
collab_ignore_items = interactions_movies_df[interactions_movies_df['userId'] == userid]['movieId'].tolist()
#collab_ignore_items
collab_recommendations_df = sorted_user_predictions[~sorted_user_predictions['movieId'].isin(collab_ignore_items)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(10)

In [95]:
collab_recommendations_df

,movieId,recStrength
3,1466,0.411030
5,2028,0.371636
6,1527,0.351813
7,1722,0.339048
8,1777,0.338855
10,1047,0.326210
11,778,0.320984
14,216,0.310144
15,2278,0.309570
16,353,0.306198


In [96]:
## Merge movies_df (with only content_id, title, text columns) with rec_df

collab_recommendations_df_1 = collab_recommendations_df.merge(movies_df[['movieId', 'title', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')

In [97]:
collab_recommendations_df_1

,movieId,recStrength,title,genres
0,1466,0.411030,Donnie Brasco (1997),Crime|Drama
1,2028,0.371636,Saving Private Ryan (1998),Action|Drama|War
2,1527,0.351813,"Fifth Element, The (1997)",Action|Adventure|Comedy|Sci-Fi
3,1722,0.339048,Tomorrow Never Dies (1997),Action|Adventure|Thriller
4,1777,0.338855,"Wedding Singer, The (1998)",Comedy|Romance
5,1047,0.326210,"Long Kiss Goodnight, The (1996)",Action|Drama|Thriller
6,778,0.320984,Trainspotting (1996),Comedy|Crime|Drama
7,216,0.310144,Billy Madison (1995),Comedy
8,2278,0.309570,Ronin (1998),Action|Crime|Thriller
9,353,0.306198,"Crow, The (1994)",Action|Crime|Fantasy|Thriller


In [98]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df_1 = interactions_movies_df.pivot(index='userId', 
                                                          columns='movieId', 
                                                          values='rating').fillna(0)

users_items_pivot_matrix_df_1.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,112623,112852,113453,114180,115617,116797,117511,117590,118696,125916
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,5.0,0.0,3.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
movie_watched = users_items_pivot_matrix_df_1[1]

In [101]:
similarity_with_other_movies = users_items_pivot_matrix_df_1.corrwith(movie_watched)  # find correlation between 1 and other movies


In [105]:
similarity_df = pd.DataFrame(data=similarity_with_other_movies, index=users_items_pivot_matrix_df_1.columns,columns=['Correlation'])

In [106]:
similarity_df.head(10)

,Correlation
movieId,
1,1.000000
2,0.110350
3,0.257602
4,0.032240
5,0.180903
6,0.361697
7,0.232298
8,-0.035042
9,0.064014


In [107]:
similarity_df.reset_index(inplace=True)
#similarity_with_other_movies.sort_values(ascending=False)#.reset_index().rename(columns={: 'recStrength'})
#similarity_with_other_movies.head()

In [110]:
similarity_df.head()

,movieId,Correlation
0,1,1.000000
1,2,0.110350
2,3,0.257602
3,4,0.032240
4,5,0.180903


In [114]:
similarity_df.sort_values(by='Correlation', inplace=True, ascending=False)

In [115]:
similarity_df.head()

,movieId,Correlation
0,1,1.000000
2039,3114,0.464813
568,802,0.448421
30,32,0.424000
82,95,0.385274


In [118]:
sim_df = similarity_df.merge(movies_df[['movieId', 'title', 'genres']], how = 'left', left_on = 'movieId',
               right_on = 'movieId')
sim_df.head()

,movieId,Correlation,title,genres
0,1,1.000000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3114,0.464813,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
2,802,0.448421,Phenomenon (1996),Drama|Romance
3,32,0.424000,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
4,95,0.385274,Broken Arrow (1996),Action|Adventure|Thriller


In [119]:
# Recommend the highest predicted rating movies that the user hasn't seen yet.
userId = 36
## get list of already interacted content ids (in order to avoid coming up in the recommendation list)
collab_ignore_items_1 = interactions_movies_df[interactions_movies_df['userId'] == userid]['movieId'].tolist()
#collab_ignore_items


In [120]:
collab_recommendations_df_1 = sim_df[~sim_df['movieId'].isin(collab_ignore_items_1)] \
                               .sort_values('Correlation', ascending = False) \
                               .head(10)

In [121]:
collab_recommendations_df_1

,movieId,Correlation,title,genres
0,1,1.000000,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,3114,0.464813,Toy Story 2 (1999),Adventure|Animation|Children|Comedy|Fantasy
2,802,0.448421,Phenomenon (1996),Drama|Romance
3,32,0.424000,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
4,95,0.385274,Broken Arrow (1996),Action|Adventure|Thriller
5,1653,0.371561,Gattaca (1997),Drama|Sci-Fi|Thriller
6,648,0.370379,Mission: Impossible (1996),Action|Adventure|Mystery|Thriller
7,788,0.363347,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi
8,6,0.361697,Heat (1995),Action|Crime|Thriller
9,1073,0.357988,Willy Wonka & the Chocolate Factory (1971),Children|Comedy|Fantasy|Musical
